# Import

In [1]:
from pynq import Overlay
from Crypto.Protocol.KDF import PBKDF2 
from Crypto.Cipher import AES
import numpy as np
import time
import os

In [2]:
%run ./linux_usb_mount.ipynb

/mnt/usb


In [3]:
%run ./AES_Utils.ipynb

In [4]:
%run ./AES_Cores.ipynb

## Hardware encrypt

In [5]:
encrypt_file_source = os.path.join(usb_path,  "Shakespeare_original.txt.encrypt")

In [6]:
encrypt_file_source = os.path.join(usb_path,  "ubuntu_original.iso.encrypt")

In [7]:
ol = Overlay("./bit/decrypt_aes256_150M_core6.bit")

In [8]:
ol.ip_dict.keys()

dict_keys(['axi_dma_read_0', 'axi_dma_read_1', 'axi_dma_read_2', 'axi_dma_read_3', 'axi_dma_write_0', 'axi_dma_write_1', 'axi_dma_write_2', 'axi_dma_write_3', 'AES_ECB_decrypt_0', 'AES_ECB_decrypt_1', 'AES_ECB_decrypt_2', 'AES_ECB_decrypt_3', 'axi_dma_read_4', 'axi_dma_read_5', 'axi_dma_write_4', 'axi_dma_write_5', 'AES_ECB_decrypt_4', 'AES_ECB_decrypt_5', 'processing_system7_0'])

In [9]:
key = mkKeyByPassword(b"Powerusb")
core_number = 6
block_size = 10 * 1024 * 1024

cores = [ AES_Core( 
    ol.__getattr__(f"AES_ECB_decrypt_{i}") ,
    ol.__getattr__(f"axi_dma_write_{i}") ,
    ol.__getattr__(f"axi_dma_read_{i}"),
    key ,
    block_size
) for i in range(core_number)]

AES_ECB_decrypt_0 self decrypt test success
AES_ECB_decrypt_1 self decrypt test success
AES_ECB_decrypt_2 self decrypt test success
AES_ECB_decrypt_3 self decrypt test success
AES_ECB_decrypt_4 self decrypt test success
AES_ECB_decrypt_5 self decrypt test success


In [10]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

Init success with 6 AES cores


In [11]:
start_time = time.time()
aes_hw.enqueue_file( encrypt_file_source )
aes_hw.start_decrypt_schedule()
end_time = time.time()
print(f"time: {end_time - start_time}")

start decrypt /mnt/usb/ubuntu_original.iso.encrypt with size 1016070160
need do 96 time
time: 123.22866725921631


In [12]:
aes_hw.list_md5()

f5416aa419c6ef1941caf0832460a12a  /mnt/usb/ubuntu_original.iso.encrypt

29ca9a17aeb497f5dd79f267649aaf71  /mnt/usb/ubuntu_original.iso.encrypt.decrypt

